In [1]:
from xgboost import XGBRegressor
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit
import seaborn
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures


from skopt.space import Real, Integer
from skopt import BayesSearchCV
import requests
import json
from entsoe import EntsoePandasClient

from skforecast.ForecasterBaseline import ForecasterEquivalentDate
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import bayesian_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from skforecast.utils import check_y

from astral.sun import sun
from astral import LocationInfo
import data

c:\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
dataset =  "../datasets/energy_dataset.csv"
df = data.load_data(dataset=dataset,daily = None)




In [2]:
df2 = data.load_data(dataset = "../energy_updated.csv")
df2 = data.preprocessing(df2)
df2.rename({"Actual Load":"total load actual"},axis=1, inplace=True)
df2.head


<bound method NDFrame.head of                            ('Biomass', 'Actual Aggregated')  \
time                                                          
2021-12-31 23:00:00+00:00                             526.0   
2022-01-01 00:00:00+00:00                             533.0   
2022-01-01 01:00:00+00:00                             524.0   
2022-01-01 02:00:00+00:00                             532.0   
2022-01-01 03:00:00+00:00                             528.0   
...                                                     ...   
2023-12-31 18:00:00+00:00                             228.0   
2023-12-31 19:00:00+00:00                             236.0   
2023-12-31 20:00:00+00:00                             244.0   
2023-12-31 21:00:00+00:00                             240.0   
2023-12-31 22:00:00+00:00                             248.0   

                           ('Fossil Brown coal/Lignite', 'Actual Aggregated')  \
time                                                                 

In [6]:

x = df2.drop("total load actual",axis = 1)
y = df2["total load actual"]

tss = TimeSeriesSplit(n_splits=2)
"""
for train,test in tss.split(df):
    trainx, testx = x.iloc[train,:], x.iloc[test,:]
    trainy, testy = y.iloc[train], y.iloc[test]
"""
#"2017-06-30 23:00:00+00:00"
#'2018-03-31 23:00:00+00:00'
end_train = "2023-03-01 23:00:00+00:00"
end_validation = '2023-06-30 23:00:00+00:00'
trainx,trainy = x.loc[: end_train, :],y.loc[: end_train]
valx,valy   = x.loc[end_train:end_validation, :],y.loc[end_train:end_validation]
testx,testy  = x.loc[end_validation:, :],y.loc[end_validation:]
#return trainx,testx,trainy,testy, end_validation

testx.head

<bound method NDFrame.head of                            ('Biomass', 'Actual Aggregated')  \
time                                                          
2023-06-30 23:00:00+00:00                             404.0   
2023-07-01 00:00:00+00:00                             416.0   
2023-07-01 01:00:00+00:00                             360.0   
2023-07-01 02:00:00+00:00                             372.0   
2023-07-01 03:00:00+00:00                             364.0   
...                                                     ...   
2023-12-31 18:00:00+00:00                             228.0   
2023-12-31 19:00:00+00:00                             236.0   
2023-12-31 20:00:00+00:00                             244.0   
2023-12-31 21:00:00+00:00                             240.0   
2023-12-31 22:00:00+00:00                             248.0   

                           ('Fossil Brown coal/Lignite', 'Actual Aggregated')  \
time                                                                 

In [3]:
#fourier encoding
def fourier_features(feature,cycle_length,order):
    result = pd.DataFrame()

    k = 2 * np.pi * feature/cycle_length
    for i in range(1,order+1):
        result[f"sin_{feature.name}_{i}"] =  np.sin(i*k)
        result[f"cos_{feature.name}_{i}"]    =  np.cos(i*k)
    return result


              


In [4]:
location = LocationInfo(
    name='Washington DC',
    region='Spain',
    timezone='CET')

calendar_features = pd.DataFrame(index=df2.index)
calendar_features['month'] = calendar_features.index.month
calendar_features['week_of_year'] = calendar_features.index.isocalendar().week
calendar_features['week_day'] = calendar_features.index.day_of_week + 1
calendar_features['hour_day'] = calendar_features.index.hour + 1
sunrise_hour = [
    sun(location.observer, date=date, tzinfo=location.timezone)['sunrise'].hour
    for date in df2.index
]
sunset_hour = [
    sun(location.observer, date=date, tzinfo=location.timezone)['sunset'].hour
    for date in df2.index
]
sun_light_features = pd.DataFrame({
                         'sunrise_hour': sunrise_hour,
                         'sunset_hour': sunset_hour}, 
                         index = df2.index
                     )
sun_light_features['daylight_hours'] = (
    sun_light_features['sunset_hour'] - sun_light_features['sunrise_hour']
)
sun_light_features['is_daylight'] = np.where(
                                        (df2.index.hour >= sun_light_features['sunrise_hour']) & \
                                        (df2.index.hour < sun_light_features['sunset_hour']),
                                        1,
                                        0
                                    )
exo_features = pd.concat([
                            calendar_features,
                            sun_light_features,
                         
                        ], axis=1)

month_encoded = fourier_features(exo_features["month"], 12,1)
week_of_year_encoded = fourier_features(exo_features['week_of_year'], 52,1)
week_day_encoded = fourier_features(exo_features['week_day'], 7,1)
hour_day_encoded = fourier_features(exo_features['hour_day'], 24,1)
cyclical_features = pd.concat([
                        month_encoded,
                        week_of_year_encoded,
                        week_day_encoded,
                        hour_day_encoded
                    ], axis=1)
exo_features = pd.concat([exo_features, cyclical_features], axis=1)
print(exo_features.head)

<bound method NDFrame.head of                            month  week_of_year  week_day  hour_day  \
time                                                                 
2021-12-31 23:00:00+00:00     12            52         5        24   
2022-01-01 00:00:00+00:00      1            52         6         1   
2022-01-01 01:00:00+00:00      1            52         6         2   
2022-01-01 02:00:00+00:00      1            52         6         3   
2022-01-01 03:00:00+00:00      1            52         6         4   
...                          ...           ...       ...       ...   
2023-12-31 18:00:00+00:00     12            52         7        19   
2023-12-31 19:00:00+00:00     12            52         7        20   
2023-12-31 20:00:00+00:00     12            52         7        21   
2023-12-31 21:00:00+00:00     12            52         7        22   
2023-12-31 22:00:00+00:00     12            52         7        23   

                           sunrise_hour  sunset_hour  dayli

In [5]:
transformer_poly = PolynomialFeatures(
                       degree           = 2,
                       interaction_only = True,
                       include_bias     = False,
                       

                   ).set_output(transform="pandas")
"""    'sin_sunrise_hour_1',
    'cos_sunrise_hour_1',
    'sin_sunset_hour_1',
    'cos_sunset_hour_1',"""
poly_cols = [
    'sin_month_1', 
    'cos_month_1',
    'sin_week_of_year_1',
    'cos_week_of_year_1',
    'sin_week_day_1',
    'cos_week_day_1',
    'sin_hour_day_1',
    'cos_hour_day_1',
    'daylight_hours',
    'is_daylight'

]

poly_features = transformer_poly.fit_transform(exo_features[poly_cols].dropna())
poly_features = poly_features.drop(columns=poly_cols)
poly_features.columns = [f"poly_{col}" for col in poly_features.columns]
poly_features.columns = poly_features.columns.str.replace(" ", "__")
exo_features = pd.concat([exo_features, poly_features], axis=1)
exo_features.head(4)

,month,week_of_year,week_day,hour_day,sunrise_hour,sunset_hour,daylight_hours,is_daylight,sin_month_1,cos_month_1,...,poly_cos_week_day_1__sin_hour_day_1,poly_cos_week_day_1__cos_hour_day_1,poly_cos_week_day_1__daylight_hours,poly_cos_week_day_1__is_daylight,poly_sin_hour_day_1__cos_hour_day_1,poly_sin_hour_day_1__daylight_hours,poly_sin_hour_day_1__is_daylight,poly_cos_hour_day_1__daylight_hours,poly_cos_hour_day_1__is_daylight,poly_daylight_hours__is_daylight
time,,,,,,,,,,,,,,,,,,,,,
2021-12-31 23:00:00+00:00,12,52,5,24,8,16,8,0,-2.449294e-16,1.000000,...,5.450191e-17,-0.222521,-1.780167,-0.0,-2.449294e-16,-1.959435e-15,-0.0,8.000000,0.0,0.0
2022-01-01 00:00:00+00:00,1,52,6,1,8,16,8,0,5.000000e-01,0.866025,...,1.613710e-01,0.602245,4.987918,0.0,2.500000e-01,2.070552e+00,0.0,7.727407,0.0,0.0
2022-01-01 01:00:00+00:00,1,52,6,2,8,16,8,0,5.000000e-01,0.866025,...,3.117449e-01,0.539958,4.987918,0.0,4.330127e-01,4.000000e+00,0.0,6.928203,0.0,0.0
2022-01-01 02:00:00+00:00,1,52,6,3,8,16,8,0,5.000000e-01,0.866025,...,4.408739e-01,0.440874,4.987918,0.0,5.000000e-01,5.656854e+00,0.0,5.656854,0.0,0.0


In [158]:
estimators = [
    ("reg", XGBRegressor())
]


search_params = {
    "reg__max_depth": Integer(2,8),
    "reg__learning_rate": Real(0.001,0.1,prior="log-uniform"),
    "reg__subsample": Real(0.5,1.0),
    "reg__reg_alpha": Real(0.0,10.0),
    "reg__reg_lambda": Real(0.0,10.0),
    "reg__gamma": Real(0.0,10.0)

}
pipe = Pipeline(steps = estimators)
opt = BayesSearchCV(pipe, search_params,cv=2,n_iter = 10, scoring = "r2")


opt.fit(trainx,trainy)


BayesSearchCV(cv=2,
              estimator=Pipeline(steps=[('reg',
                                         XGBRegressor(base_score=None,
                                                      booster=None,
                                                      callbacks=None,
                                                      colsample_bylevel=None,
                                                      colsample_bynode=None,
                                                      colsample_bytree=None,
                                                      device=None,
                                                      early_stopping_rounds=None,
                                                      enable_categorical=False,
                                                      eval_metric=None,
                                                      feature_types=None,
                                                      gamma=None,
                                                      grow_policy=None,
                                                      importance_type=None,
                                                      interaction_constraints=N...
                             'reg__learning_rate': Real(low=0.001, high=0.1, prior='log-uniform', transform='normalize'),
                             'reg__max_depth': Integer(low=2, high=8, prior='uniform', transform='normalize'),
                             'reg__reg_alpha': Real(low=0.0, high=10.0, prior='uniform', transform='normalize'),
                             'reg__reg_lambda': Real(low=0.0, high=10.0, prior='uniform', transform='normalize'),
                             'reg__subsample': Real(low=0.5, high=1.0, prior='uniform', transform='normalize')})

In [ ]:
opt.best_score_
opt.cv_results_

In [160]:
opt.score(testx,testy)

0.8864712914872862

In [ ]:
testx.columns
predy = opt.predict(testx)
fig, ax = plt.subplots(figsize=[20,5])


#data = np.vstack((predy[0:7],testy.iloc[0:7].to_numpy()))
predicted = pd.DataFrame(data = predy,index = testx.index,columns=["predicted load"])
#predicted.head
data = pd.concat([testy,predicted],axis=1)
#print(predicted.head)
#print(testy.iloc[0])
print(data.head)
timespan = {"week":7}
#seaborn.lineplot(ax = ax, x = testx.index[0:7],y = data)
seaborn.lineplot(ax = ax, data = data[0:365])
#print(predy[0:7])

In [ ]:
#skforecast
forecast = ForecasterAutoreg(regressor = XGBRegressor(random_state = 1543),lags = 10)
forecast.fit(y=trainy)
forecast


In [14]:
metric, predictions = backtesting_forecaster(
                          forecaster         = forecast,
                          y                  = df2["total load actual"],
                          steps              = 7,
                          metric             = 'mean_absolute_percentage_error',
                          initial_train_size = len(trainy),
                          refit              = False,
                          n_jobs             = 'auto',
                          verbose            = True, # Change to False to see less information
                          show_progress      = True
                      )

predictions.head()

Information of backtesting process
----------------------------------
Number of observations used for initial training: 10201
Number of observations used for backtesting: 7319
    Number of folds: 1046
    Number of steps per fold: 7
    Number of steps to exclude from the end of each train set before test (gap): 0
    Last fold only includes 4 observations.

Fold: 0
    Training:   2021-12-31 23:00:00+00:00 -- 2023-03-01 23:00:00+00:00  (n=10201)
    Validation: 2023-03-02 00:00:00+00:00 -- 2023-03-02 06:00:00+00:00  (n=7)
Fold: 1
    Training:   2021-12-31 23:00:00+00:00 -- 2023-03-01 23:00:00+00:00  (n=10201)
    Validation: 2023-03-02 07:00:00+00:00 -- 2023-03-02 13:00:00+00:00  (n=7)
Fold: 2
    Training:   2021-12-31 23:00:00+00:00 -- 2023-03-01 23:00:00+00:00  (n=10201)
    Validation: 2023-03-02 14:00:00+00:00 -- 2023-03-02 20:00:00+00:00  (n=7)
Fold: 3
    Training:   2021-12-31 23:00:00+00:00 -- 2023-03-01 23:00:00+00:00  (n=10201)
    Validation: 2023-03-02 21:00:00+00:00 --

100%|██████████| 1046/1046 [00:03<00:00, 293.10it/s]


,pred
2023-03-02 00:00:00+00:00,26091.246094
2023-03-02 01:00:00+00:00,24884.556641
2023-03-02 02:00:00+00:00,24209.814453
2023-03-02 03:00:00+00:00,23877.984375
2023-03-02 04:00:00+00:00,24134.087891


In [15]:
print(f'Backtest error (MAE): {metric}')

Backtest error (MAE): 0.0228532380977935


In [13]:
lags_grid = [48, 72, [1, 2, 3, 23, 24, 25, 167, 168, 169]]

# Regressor hyperparameters search space
def search_space(trial):
    search_space  = {
        'n_estimators'  : trial.suggest_int('n_estimators', 400, 1200, step=100),
        'max_depth'     : trial.suggest_int('max_depth', 3, 10, step=1),
        'learning_rate' : trial.suggest_float('learning_rate', 0.01, 0.5),
        'reg_alpha'     : trial.suggest_float('reg_alpha', 0, 1, step=0.1),
        'reg_lambda'    : trial.suggest_float('reg_lambda', 0, 1, step=0.1),
    } 
    return search_space
search_params = {
    "reg__max_depth": Integer(2,8),
    "reg__learning_rate": Real(0.001,0.1,prior="log-uniform"),
    "reg__subsample": Real(0.5,1.0),
    "reg__reg_alpha": Real(0.0,10.0),
    "reg__reg_lambda": Real(0.0,10.0),
    "reg__gamma": Real(0.0,10.0)

}

results_search, frozen_trial = bayesian_search_forecaster(
                                   forecaster         = forecast,
                                   y                  = y.loc[:end_validation], # Test data not used
                                   search_space       = search_space,
                                   lags_grid          = lags_grid,
                                   steps              = 36,
                                   refit              = False,
                                   metric             = 'mean_absolute_error',
                                   initial_train_size = len(trainy),
                                   fixed_train_size   = False,
                                   n_trials           = 10, # Increase this value for a more exhaustive search
                                   random_state       = 123,
                                   return_best        = True,
                                   n_jobs             = 'auto',
                                   verbose            = False,
                                   show_progress      = True
                               )

Number of models compared: 30,
         10 bayesian search in each lag configuration.


lags grid: 100%|██████████| 3/3 [02:54<00:00, 58.31s/it]


`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [  1   2   3  23  24  25 167 168 169] 
  Parameters: {'n_estimators': 1000, 'max_depth': 5, 'learning_rate': 0.12115721224645952, 'reg_alpha': 0.6000000000000001, 'reg_lambda': 0.7000000000000001}
  Backtesting metric: 926.7714194726025



In [10]:
features = []
features.extend(exo_features.filter(regex='sin_|cos_').columns.tolist())
exo_features =  exo_features.filter(features, axis=1)
df3 = pd.DataFrame
"""df2 = df2["total load actual"].merge(exo_features,
           left_index=True,
           right_index=True,
           how='left')"""

df3 = pd.concat([df2["total load actual"],exo_features])
df3.rename(columns={0: 'total load actual'}, inplace=True)

exo_features["total actual load"] = df2["total load actual"]
exo_features.columns

Index(['sin_month_1', 'cos_month_1', 'sin_week_of_year_1',
       'cos_week_of_year_1', 'sin_week_day_1', 'cos_week_day_1',
       'sin_hour_day_1', 'cos_hour_day_1', 'poly_sin_month_1__cos_month_1',
       'poly_sin_month_1__sin_week_of_year_1',
       'poly_sin_month_1__cos_week_of_year_1',
       'poly_sin_month_1__sin_week_day_1', 'poly_sin_month_1__cos_week_day_1',
       'poly_sin_month_1__sin_hour_day_1', 'poly_sin_month_1__cos_hour_day_1',
       'poly_sin_month_1__daylight_hours', 'poly_sin_month_1__is_daylight',
       'poly_cos_month_1__sin_week_of_year_1',
       'poly_cos_month_1__cos_week_of_year_1',
       'poly_cos_month_1__sin_week_day_1', 'poly_cos_month_1__cos_week_day_1',
       'poly_cos_month_1__sin_hour_day_1', 'poly_cos_month_1__cos_hour_day_1',
       'poly_cos_month_1__daylight_hours', 'poly_cos_month_1__is_daylight',
       'poly_sin_week_of_year_1__cos_week_of_year_1',
       'poly_sin_week_of_year_1__sin_week_day_1',
       'poly_sin_week_of_year_1__cos_wee

In [7]:
#model trained using exogenous features
features = []
features.extend(exo_features.filter(regex='sin_|cos_').columns.tolist())
exo_features =  exo_features.filter(features, axis=1)
df3 = pd.DataFrame
"""df2 = df2["total load actual"].merge(exo_features,
           left_index=True,
           right_index=True,
           how='left')"""

df3 = pd.concat([df2["total load actual"],exo_features])
df3.rename(columns={0: 'total load actual'}, inplace=True)

exo_features["total actual load"] = df2["total load actual"]


traintrainx,testx,trainy,testy, end_validation = data.split_data(exo_features,"total actual load","2023-03-01 23:00:00+00:00",'2023-06-30 23:00:00+00:00')

forecast = ForecasterAutoreg(regressor = XGBRegressor(random_state = 1543),lags = 10)
forecast.fit(y=trainy)

# Lags grid
lags_grid = [[1, 2, 3, 23, 24, 25, 167, 168, 169]]

# Regressor hyperparameters search space
def search_space(trial):
    search_space  = {
        'n_estimators' : trial.suggest_int('n_estimators', 800, 1400, step=100),
        'max_depth'    : trial.suggest_int('max_depth', 3, 8, step=1),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5),
        'reg_alpha'    : trial.suggest_float('reg_alpha', 0, 1, step=0.1),
        'reg_lambda'   : trial.suggest_float('reg_lambda', 0, 1, step=0.1),
    } 
    return search_space

results_search, frozen_trial = bayesian_search_forecaster(
                                   forecaster         = forecast,
                                   y                  = exo_features.loc[:end_validation, "total actual load"],
                                   exog               = exo_features.loc[:end_validation, exo_features.columns!="total actual load"],
                                   search_space       = search_space,
                                   lags_grid          = lags_grid,
                                   steps              = 36,
                                   refit              = False,
                                   metric             = 'mean_absolute_error',
                                   initial_train_size = len(trainy),
                                   fixed_train_size   = False,
                                   n_trials           = 20,
                                   random_state       = 123,
                                   return_best        = True,
                                   n_jobs             = 'auto',
                                   verbose            = False,
                                   show_progress      = True
                               )

Number of models compared: 20,
         20 bayesian search in each lag configuration.


lags grid: 100%|██████████| 1/1 [01:38<00:00, 98.64s/it]


`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [  1   2   3  23  24  25 167 168 169] 
  Parameters: {'n_estimators': 1400, 'max_depth': 3, 'learning_rate': 0.018066668056670537, 'reg_alpha': 0.2, 'reg_lambda': 0.0}
  Backtesting metric: 911.0635915708935



In [ ]:
predicted2 = forecast.predict(steps=10)
predicted2.index = testy.iloc[:10].index
#predicted2 = predicted2.index(testy.iloc[:10].index)
print(predicted2)
#print(testy.iloc[:10])
data2 = pd.concat([testy.iloc[:10],predicted2],axis=1)
print(data2)

fig,ax = plt.subplots(figsize=[20,5])
seaborn.lineplot(ax = ax, data = data2)
